In [3]:
import torch
import torch.nn as nn
import numpy as np
from mmpretrain import get_model
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from sklearn.cluster import KMeans
from torch.utils.data import DataLoader

torch.cuda.set_device(7)


# Load ResNet50 model and remove the final classification layer
def load_resnet50_model(model_pth):
    pre_trained_ResNet50 = model_pth
    # Initialize SAR pre-trained ResNet50
    model = get_model(
        "resnet50_8xb32_in1k",
        head=None,  # to extract only activation vectors
        pretrained=pre_trained_ResNet50,
    ).cuda()
    model = model.eval()

    return model


# Extract features from images using ResNet50
def extract_features(dataloader, model):
    features_list = []

    with torch.no_grad():
        for inputs, _ in dataloader:
            inputs = inputs.cuda()
            features = model(inputs)[0]
            # features: batch_size x 2048 (before FC layer but after GAP)
            features = features.reshape(features.size(0), -1)
            features_list.append(features.cpu().numpy())

    return np.vstack(features_list)


# Load dataset
data_transform = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[11.20954390854399, 11.20954390854399, 11.20954390854399],
            std=[20.241805767392393, 20.241805767392393, 20.241805767392393],
        ),
    ]
)

dataset = ImageFolder(
    root="/workspace/data/fusrs_v2/vgg_format", transform=data_transform
)
dataloader = DataLoader(dataset, batch_size=32, shuffle=False, num_workers=4)

# Extract features using ResNet50
model = load_resnet50_model(
    model_pth="/workspace/dso/clsar/outputs/res50_fusrs_v2_pretrain/res50_1x128_lr1e-1+200e+im21k_fusrs_v2/best_f1_score_epoch_158.pth"
)
features = extract_features(dataloader, model)
print(f"features shape: {features.shape}")

# Cluster features using K-means
kmeans = KMeans(n_clusters=4, random_state=0).fit(features)
labels = kmeans.predict(features)

# Create a dictionary mapping from original image paths to new labels
path_to_label = {path: label for path, label in zip(dataset.samples, labels)}


# Optionally, you can save this mapping to a file for future reference
with open("./cluster_labels_c4.txt", "w") as f:
    for path, label in path_to_label.items():
        f.write(f"{path[0]},{path[1]},{label}\n")

print("Clustering done and labels saved!")


# Extract the original labels from the dataset
original_labels = [label for _, label in dataset.samples]

# Create a matrix to store counts
# num_classes: the total number of original classes
num_classes = len(dataset.classes)
confusion_matrix = np.zeros((num_classes, 4), dtype=int)

# Populate the confusion matrix
for original_label, cluster_label in zip(original_labels, labels):
    confusion_matrix[original_label, cluster_label] += 1

# Print the confusion matrix
print("Confusion matrix: (Original classes x New clusters)")
print(confusion_matrix)

# Optionally, you can also print the matrix with class names for clarity
for original_class, row in zip(dataset.classes, confusion_matrix):
    print(f"{original_class}: {row}")

Loads checkpoint by local backend from path: /workspace/dso/clsar/outputs/res50_fusrs_v2_pretrain/res50_1x128_lr1e-1+200e+im21k_fusrs_v2/best_f1_score_epoch_158.pth
The model and loaded state dict do not match exactly

unexpected key in source state_dict: head.fc.weight, head.fc.bias

features shape: (6848, 2048)


/opt/conda/envs/mmpretrain/lib/python3.10/site-packages/sklearn/cluster/_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


Clustering done and labels saved!
Confusion matrix: (Original classes x New clusters)
[[1513  153  417 1299]
 [ 171    2    8   91]
 [ 354  149   45  368]
 [ 876  128   76  843]
 [ 239    4   23   89]]
cargo: [1513  153  417 1299]
dredger: [171   2   8  91]
fishing: [354 149  45 368]
other: [876 128  76 843]
tanker: [239   4  23  89]


In [18]:
from collections import defaultdict

# Read the image_labels.txt file to get the mappings
with open("cluster_labels.txt", "r") as f:
    lines = f.readlines()

# Extract path, original label, and new cluster label from each line
path_to_labels = {
    line.split(",")[0]: (int(line.split(",")[1]), int(line.split(",")[2]))
    for line in lines
}

# Determine the distribution of original classes within each cluster
cluster_to_class_distribution = defaultdict(lambda: defaultdict(int))

# Extract path, original label, and new cluster label from each line
path_to_labels = {
    line.split(",")[0]: (int(line.split(",")[1]), int(line.split(",")[2]))
    for line in lines
}

for path, (original_label, cluster) in path_to_labels.items():
    cluster_to_class_distribution[cluster][original_label] += 1

# Print the distribution for each cluster
for cluster, class_distribution in cluster_to_class_distribution.items():
    print(f"Cluster {cluster}:")
    for original_class, count in class_distribution.items():
        print(f"  Original Class {original_class}: {count}")
    print()

Cluster 11:
  Original Class 0: 445
  Original Class 1: 75
  Original Class 2: 44
  Original Class 3: 139
  Original Class 4: 44

Cluster 3:
  Original Class 0: 583
  Original Class 1: 82
  Original Class 2: 64
  Original Class 3: 246
  Original Class 4: 171

Cluster 1:
  Original Class 0: 415
  Original Class 1: 17
  Original Class 2: 45
  Original Class 3: 89
  Original Class 4: 24

Cluster 4:
  Original Class 0: 264
  Original Class 1: 20
  Original Class 2: 30
  Original Class 3: 86
  Original Class 4: 23

Cluster 5:
  Original Class 0: 307
  Original Class 1: 29
  Original Class 2: 70
  Original Class 3: 68
  Original Class 4: 27

Cluster 0:
  Original Class 0: 162
  Original Class 1: 7
  Original Class 2: 18
  Original Class 3: 15
  Original Class 4: 26

Cluster 14:
  Original Class 0: 475
  Original Class 1: 14
  Original Class 2: 62
  Original Class 3: 174
  Original Class 4: 22

Cluster 7:
  Original Class 0: 196
  Original Class 1: 1
  Original Class 2: 95
  Original Class 3:

In [19]:
import numpy as np
from collections import defaultdict

# Read the image_labels.txt file to get the mappings
with open("cluster_labels.txt", "r") as f:
    lines = f.readlines()

# Extract path, original label, and new cluster label from each line
path_to_labels = {
    line.split(",")[0]: (int(line.split(",")[1]), int(line.split(",")[2].strip()))
    for line in lines
}

# Create a matrix for the distribution
# Note: This assumes original labels are sequential integers starting from 0.
# Adjust max_original_label and num_clusters accordingly if different.
max_original_label = max([original for original, _ in path_to_labels.values()]) + 1
num_clusters = 16
distribution_matrix = np.zeros((max_original_label, num_clusters))

for _, (original_label, cluster) in path_to_labels.items():
    distribution_matrix[original_label][cluster] += 1

# Print the distribution matrix
print("Distribution Matrix:")
print(distribution_matrix)

# If you want a more detailed print:
for i, row in enumerate(distribution_matrix):
    print(f"Original Class {i}:", end=" ")
    for j, count in enumerate(row):
        print(f"Cluster {j}: {count}", end=" | ")
    print()

Distribution Matrix:
[[162. 415.  52. 583. 264. 307.   6. 196.  34.  47. 169. 445.   2. 224.
  475.   1.]
 [  7.  17.   3.  82.  20.  29.   0.   1.   1.   0.  10.  75.   0.  13.
   14.   0.]
 [ 18.  45.  23.  64.  30.  70.  19.  95. 122.  39.  74.  44.   0. 209.
   62.   2.]
 [ 15.  89.  11. 246.  86.  68.  14. 130. 396.  44.  68. 139.  13. 423.
  174.   7.]
 [ 26.  24.   0. 171.  23.  27.   0.   3.   1.   3.   2.  44.   0.   9.
   22.   0.]]
Original Class 0: Cluster 0: 162.0 | Cluster 1: 415.0 | Cluster 2: 52.0 | Cluster 3: 583.0 | Cluster 4: 264.0 | Cluster 5: 307.0 | Cluster 6: 6.0 | Cluster 7: 196.0 | Cluster 8: 34.0 | Cluster 9: 47.0 | Cluster 10: 169.0 | Cluster 11: 445.0 | Cluster 12: 2.0 | Cluster 13: 224.0 | Cluster 14: 475.0 | Cluster 15: 1.0 | 
Original Class 1: Cluster 0: 7.0 | Cluster 1: 17.0 | Cluster 2: 3.0 | Cluster 3: 82.0 | Cluster 4: 20.0 | Cluster 5: 29.0 | Cluster 6: 0.0 | Cluster 7: 1.0 | Cluster 8: 1.0 | Cluster 9: 0.0 | Cluster 10: 10.0 | Cluster 11: 75.0 | Clu

In [4]:
from collections import defaultdict

# Read the image_labels.txt file
with open("cluster_labels_c4.txt", "r") as f:
    lines = f.readlines()

# Extract path, original label, and new cluster label from each line
path_to_labels = {
    line.split(",")[0]: (int(line.split(",")[1]), int(line.split(",")[2]))
    for line in lines
}

# Determine the distribution of original classes within each cluster
class_to_cluster_distribution = defaultdict(lambda: defaultdict(int))

for path, (original_label, cluster) in path_to_labels.items():
    class_to_cluster_distribution[original_label][cluster] += 1

# Identify unique original classes and clusters for matrix headers
all_original_classes = sorted(class_to_cluster_distribution.keys())
all_clusters = sorted(
    set(
        cluster
        for class_distr in class_to_cluster_distribution.values()
        for cluster in class_distr.keys()
    )
)

# Print the matrix
# Header
print("\t" + "\t".join(f"C{c}" for c in all_clusters))
print("-" * (8 + 10 * len(all_clusters)))

# Rows
for original_class in all_original_classes:
    counts = [
        str(class_to_cluster_distribution[original_class].get(cluster, 0))
        for cluster in all_clusters
    ]
    print(f"Class {original_class}\t" + "\t".join(counts))

	C0	C1	C2	C3
------------------------------------------------
Class 0	1513	153	417	1299
Class 1	171	2	8	91
Class 2	354	149	45	368
Class 3	876	128	76	843
Class 4	239	4	23	89
